# Prototyping Wearable Firmware with Espruino
The following presents introductory code examples for the [Espruino](https://www.espruino.com) [Bangle](https://www.banglejs.com) wearables.
To run the code, use [Espruino IDE](https://www.espruino.com/ide) and copy-paste the code cells below, without the `%%javascript` header line, into the right pane. Upload the code to RAM then via the emulator or WebBLE (middle button).

This lesson has several important take-home messages that show the advantages and challenges of this type of prototyping. For clarity, these have been put in bold.

Let's start with a small example that buzzes the user to stay awake every 4 seconds, using just a few classes such as [Bangle](https://www.espruino.com/Reference#Bangle) and the [Graphics](https://www.espruino.com/Reference#Graphics) class object g:

In [8]:
%%javascript
/* initialize a blink variable to true */
var blink = true;
/* clear the graphics context and setup the font and its alignment  */
g.clear().setFont("Vector:27").setFontAlign(0, 0);
/* use an interval of 2 seconds to buzz every 4 seconds */
setInterval(function() {
    g.clear();
    if (blink) {
      g.drawString("STAY AWAKE", g.getWidth()/2, 60);
      Bangle.buzz();  /* make the Bangle buzz */
    }
    blink = !blink;
  }, 2000);

<IPython.core.display.Javascript object>

The Bangle JS watches interpret the JS code that is sent via WebBLE locally, on the microcontroller. This will have a small overhead, but the rich libraries and short code are perfect for prototyping.

**Take-away message 1**: Javascript code can run straight on the wearable's microcontroller, editable through an online IDE that connects through WebBLE to the Bangle. This results in a fast and interactive development process.



This approach has consequences. To make uploading the code faster for example, code can be shortened and minimized.
A shorter version is possible with the JS lambda notation, where `() => (a, b, c, ...)` uses the comma operator to execute multiple statements, using the ternary operator `a?b:c` and leaving out the semicolons (allowed in JS):  

In [6]:
%%javascript
blink = 1
g.clear().setFont("Vector:27").setFontAlign(0, 0)
setInterval( () => (
    g.clear(),
    blink ? g.drawString("STAY AWAKE", g.getWidth()/2, 60) && Bangle.buzz() : 0,
    blink = !blink
  ), 2000)

<IPython.core.display.Javascript object>

Further minifying (removing spaces and comments) the JS code will make it shorter. The script below shows you for any example in this notebook what its size is in characters, before and after minifying. Run the JS code cell first, then the script below:

In [9]:
# Script to count JS code characters. Run the JS-cell first, then this code.
import importlib.util
if importlib.util.find_spec("jsmin") is None:
    !pip install -q jsmin
    from jsmin import jsmin
try:
  if str(In[-2]).startswith("get_ipython().run_cell_magic('javascript', '', '"):
    jscode = In[-2][48:-2];  #print(repr(jscode))
    print(f"JS character count: {len(jscode)}, minified: {len(jsmin(jscode))}");
  else: print("The last run code was not JS code.")
except NameError: print("Execute a JS cell first.")

JS character count: 452, minified: 221


**Take-away message 2:**
Interpreting javascript code on the wearable makes coding easier, but it comes at the cost of having to send (or locally store) the code as a string that needs parsing first. This can get out of hand for larger projects.

Check the two javascript code cells above to see the difference that JS code design and minifying makes.

Note that mangling (shortening) of names and pretokenisation will make this even more efficient and are built in the Espruino online editor.

Even though JS code interpreted locally is surprisingly efficient, there are several options to implement a function more efficiently:
1.   Adding "ram"; as the first line of the function will place the function pretokenized in RAM, so it loads faster
2.   Adding "compiled"; as the first line of the function will send it to the Espruino server and compile with c++ to the target platform
3.   Adding "jit"; as the first line of the function will run a local compiler to do the same
4.   using the [E.compiledC()](https://www.espruino.com/Reference#l_E_compiledC) function will generate an inline C function



Running the following code on an actual Espruino platform (results on emulators will be skewed) shows the differences between these options.

In [ ]:
%%javascript

// normal javascript function:
function fun(n) { let sum=0.0; for (i = 0; i < n; i++) sum += i+0.1; }

// in-RAM and pretokenized function:
function ram_fun(n) { "ram"; let sum=0.0; for (i=0; i<n; i++) sum += i+0.1; }

// c++ compiled code on the Espruino server, into a binary of native code:
function cmp_fun(n) { "compiled"; let sum=0.0; for (i=0; i<n; i++) sum += i+0.1;}

// JIT (https://www.espruino.com/JIT) function:
function jit_fun(n) { "jit"; let sum=0.0; for (i=0; i<n; i++) sum += i+0.1; }

// inline C function:
var c = E.compiledC(`// void fun(int)
void fun(int n){ float sum=0.0; for (int i=0; i<n; i++) sum+=(float)i+0.1; }`);

// time and show performance with helper function:
function timeFn(name, fn) {
  t = getTime(); for (i=0; i<9; i++) fn(900); print(name, (getTime()-t)/9);
}
timeFn("normal   ", fun);      timeFn("ram      ", ram_fun);
timeFn("compiled ", cmp_fun);  timeFn("jit      ", jit_fun);
timeFn("inline C ", c.fun);

<IPython.core.display.Javascript object>

On a Bangle v2, this returned:
```
normal    0.04117838541
ram       0.04083930121
compiled  0.01759507921
jit       0.01822916666
inline C  0.00038994683
```

What makes javascript such a fast-to-learn scripting language also helps quick prototyping for the Bangle JS.
Below is a short example that implements a basic stopwatch App. It illustrates three strengths of using javascript to prototype such a low-level smart watch application.

Note that the [graphics class](https://www.espruino.com/Reference#Graphics) object `g` represents the smart watch display, and its methods return the object again. `setInterval()` is used to trigger function calls at a regular frequency (every 90ms). The `setWatch()` function links a (lambda) function to a button (`BTN1`) to be executed each time the button is pressed (`rising` `edge`).

**Take-away message 3:** Code simplicity is obtained by wrapping up low-level concepts such as timers using javascript SetInterval(), or objects in libraries to drive displays or sensors.

In [ ]:
%%javascript

var startTime, elapsedTime=0, timer, state = false;

function update() {
  if (state) elapsedTime = Date.now() - startTime;
  g.clear().setFont("Vector:25").setFontAlign(0, 0);
  g.drawString("Stopwatch", g.getWidth()/2, 20);
  secs = Math.floor(elapsedTime / 1000);
  mins = Math.floor(secs / 60);
  secs = secs % 60; ms = elapsedTime % 1000;
  str = mins + ":" + ("0"+secs).slice(-2) + "." + ("00"+ms).slice(-3);
  g.drawString(str, g.getWidth()/2, 50);
}

setWatch( () => {
    if (!state) {
      startTime = Date.now(); timer = setInterval(update, 90); // 90ms
    } else {
      clearInterval(timer); elapsedTime = 0; update();
    }
    state=!state;
  }, BTN1, {repeat:true, edge:"rising"});
update();

A final illustration of the quick prototyping capabilities shows how easy it is to set up a broadcast of sensor data to all nearby devices. For this, the bluetooth low energy (BLE) tranceiver is used through the `NRF` class (for Nordic Radio Frequency). Although BLE communications usually go between two devices that are paired, the BLE advertisements can be used to transmit a few bytes to any nearby devices.

The following code works on the Bangle watches and packs the measured heart rate value in the BLE advertisement. The Bangle.on() method links an event (heart rate measurement or HRM) to the supplied lambda function, so that this is updated with every new heart rate measurement.

In [ ]:
%%javascript

Bangle.setHRMPower(1);

Bangle.on('HRM', (hrm) => {
    // Encode heart rate in advertisement [0x90, 0x05, 0x00, heartRate, 0x00]
    let advData = {
      manufacturer: 0x0590, // Bangle.js vendor ID
      manufacturerData: [0, hrm.bpm, 0]  // Heart rate in 1 byte (0-255 BPM)
    };
    NRF.setAdvertising({}, advData);
    g.clear().setFont("Vector:25").setFontAlign(0, 0);
    g.drawString(hrm.bpm+"(0x"+ hrm.bpm.toString(16).toUpperCase()+")", g.getWidth()/2, 60);
  });

You can use a BLE scanning application such as [NRF Connect](https://apps.apple.com/us/app/nrf-connect-for-mobile/id1054362403) or [lightBlue](https://apps.apple.com/de/app/lightblue/id557428110) to inspect the advertisement packages that the Bangle is broadcasting.

Similarly, dedicated communication channels can be set up to link the Bangle wearable to other devices and exchange information.

**Take-away message 4:** Built-in sensor and BLE capabilities can be accessed quickly through a dedicated class to prototype otherwise complex behaviors. BLE Advertisements are an ad-hoc solution to broadcasting data.


More information can be found here:
*   App loader and Apps by others: https://banglejs.com/apps
*   More tutorials: https://www.espruino.com/Tutorials
*   A first tutorial on creating a watch App:
https://www.espruino.com/Bangle.js+Clock
